In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
import time
from typing import Optional 

In [3]:
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, api_key=os.getenv("ANTHROPIC_API_KEY"))

In [48]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters. If there are multiple volumes, focus on the first volume.")

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [6]:
story_info = {
    "title": "Pride and Prejudice",
    "type": "Book",
    "author": "Jane Austen",
    "year": 1813,
    "url": "https://www.gutenberg.org/ebooks/1342",
    "image": "https://m.media-amazon.com/images/I/51oQBC68VSL.jpg",
}

In [7]:
# read in txt file from scripts folder
story_name = "pride"
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [46]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]

In [47]:
# split story into 500000 character chunks
print(len(story))
story_chunks = [story[i:i + 500000] for i in range(0, len(story), 500000)]
print(len(story_chunks))

232777
1


In [11]:
all_res = []

In [49]:
# for i, chunk in enumerate(story_chunks):
#     invalid_input = True
#     while invalid_input: # try until valid input
#         # see if exception is thrown
#         try:
#             res = structured_llm.invoke(chunk)
#             all_res.append(res)
#             print(f"Chunk {i} done.")
#             invalid_input = False
#         except:
#             print("Exception thrown. Trying again.")
#             invalid_input = True

#     if i < len(story_chunks) - 1:
#         # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
#         time.sleep(120)  # Adjust the sleep duration as per your rate limit requirements

invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = structured_llm.invoke(story)
        invalid_input = False
    except:
        print("Exception thrown. Trying again.")
        invalid_input = True

In [50]:
# first_line = all_res[0].firstline
# last_line = all_res[-1].lastline
# markers = [marker for res in all_res for marker in res.markers]

first_line = res.firstline
last_line = res.lastline
markers = res.markers

In [51]:
# remove any marker that doesn't include one of these words
exclude_words = ["part", "preface", "contents", "by", "end"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [52]:
markers

['CHAPTER I.',
 'CHAPTER II.',
 'CHAPTER III.',
 'CHAPTER IV.',
 'CHAPTER V.',
 'CHAPTER VI.',
 'CHAPTER VII.',
 'CHAPTER VIII.',
 'CHAPTER IX.',
 'CHAPTER X.',
 'CHAPTER XI.',
 'CHAPTER XII.',
 'CHAPTER XIII.',
 'CHAPTER XIV.',
 'CHAPTER XV.',
 'CHAPTER XVI.',
 'CHAPTER XVII.',
 'CHAPTER XVIII.',
 'CHAPTER XIX.',
 'CHAPTER XX.',
 'CHAPTER XXI.',
 'CHAPTER XXII.',
 'CHAPTER XXIII.']

In [53]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")
with open(f"json/{story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [54]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [55]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [10]:
# read in markers from summary.json
with open(f"json/{story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [62]:
story_formatted[-100:]

'hould _he_ have it more than anybody else?   I leave it to yourself to determine,  said Mr. Bennet. '

In [61]:
story_formatted.find(last_line)
last_line

'I leave it to yourself to determine.'

In [94]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    cur_last_line = f"{markers[i+1]} " if i+1 < len(markers) else last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    first_line_index = story_formatted.find(cur_first_line, cur_min_index)
    if i != 0:
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()
    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

In [95]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter CHAPTER I.
first_line: LINE 1: It is a truth universally acknowledged, that a single man in possession
LINE 2: of a good fo
last_line: he business of her life was to get her
LINE 80: daughters married; its solace was visiting and news.


Chapter CHAPTER II.
first_line: LINE 1: Mr. Bennet was among the earliest of those who waited on Mr. Bingley. He
LINE 2: had always 
last_line: uld
LINE 75: return Mr. Bennet's visit, and determining when they should ask him to
LINE 76: dinner.


Chapter CHAPTER III.
first_line: LINE 1: Not all that Mrs. Bennet, however, with the assistance of her five
LINE 2: daughters, could 
last_line: had been there, my
LINE 145: dear, to have given him one of your set downs. I quite detest the man."


Chapter CHAPTER IV.
first_line: LINE 1: When Jane and Elizabeth were alone, the former, who had been cautious in
LINE 2: her praise 
last_line: t girl, and their brother felt authorised by such
LINE 92: commendation to think of her as he chose.


Chapter CHAP

In [96]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{story_name}"):
    os.makedirs(f"chapters/{story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter CHAPTER I. saved.
Chapter CHAPTER II. saved.
Chapter CHAPTER III. saved.
Chapter CHAPTER IV. saved.
Chapter CHAPTER V. saved.
Chapter CHAPTER VI. saved.
Chapter CHAPTER VII. saved.
Chapter CHAPTER VIII. saved.
Chapter CHAPTER IX. saved.
Chapter CHAPTER X. saved.
Chapter CHAPTER XI. saved.
Chapter CHAPTER XII. saved.
Chapter CHAPTER XIII. saved.
Chapter CHAPTER XIV. saved.
Chapter CHAPTER XV. saved.
Chapter CHAPTER XVI. saved.
Chapter CHAPTER XVII. saved.
Chapter CHAPTER XVIII. saved.
Chapter CHAPTER XIX. saved.
Chapter CHAPTER XX. saved.
Chapter CHAPTER XXI. saved.
Chapter CHAPTER XXII. saved.
Chapter CHAPTER XXIII. saved.


## analyze scene

In [16]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Title of scene (don't include chapter name or number)")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

scene_llm = llm.with_structured_output(SceneListInfo)

In [17]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Location of the scene")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.")

scene_details_llm = llm.with_structured_output(SceneDetails)

In [18]:
def get_scene_info(chapters, chapter_num):
    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            chapter_scenes = scene_llm.invoke(cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            

    chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }
        formatted_chapter_scenes.append(formatted_scene)
        formatted_chapter = cur_chapter.replace("\n", " ")
        # replace weird quotes too
        formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [19]:
def get_scene_details(scenes, formatted_chapter_scenes):
    all_scene_details = []
    for i, scene in enumerate(scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        if len(scene_text) == 0:
            all_scene_details.append([])
            continue
        # get scene details for each scene
        cur_scene = scene
        scene_details = scene_details_llm.invoke(cur_scene)
        all_scene_details.append(scene_details)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []
        for character in cur_scene_details.characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote
            }
            formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [97]:
# get scene info for each chapter
for i, marker in enumerate(markers):
    if i != len(markers) - 1:
        continue

    print(f"Chapter {marker} start.")
    scenes, formatted_chapter_scenes = get_scene_info(chapters, i)
    print(f"Chapter {marker} scenes done.")
    formatted_chapter_scenes = get_scene_details(scenes, formatted_chapter_scenes)
    print(f"Chapter {marker} details done.")

    # remove scenes with no text
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
    # remove characters with no quote
    for scene in formatted_chapter_scenes:
        scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"]]
        # print(scene)
    
    # remove scenes with no characters
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(formatted_chapter_scenes, f, indent=4)

    print(f"Chapter {marker} done.\n")

Chapter CHAPTER XXIII. start.
FIRST LINE: 1
LAST LINE: 39

FIRST LINE: 40
LAST LINE: 44

FIRST LINE: 45
LAST LINE: 50

FIRST LINE: 51
LAST LINE: 55

FIRST LINE: 56
LAST LINE: 62

FIRST LINE: 63
LAST LINE: 77

FIRST LINE: 78
LAST LINE: 86

FIRST LINE: 87
LAST LINE: 100

FIRST LINE: 101
LAST LINE: 109

FIRST LINE: 110
LAST LINE: 116

FIRST LINE: 117
LAST LINE: 143

Chapter CHAPTER XXIII. scenes done.
Chapter CHAPTER XXIII. details done.
Chapter CHAPTER XXIII. done.



In [98]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

CHAPTER I.
CHAPTER II.
CHAPTER III.
CHAPTER IV.
CHAPTER V.
CHAPTER VI.
CHAPTER VII.
CHAPTER VIII.
CHAPTER IX.
CHAPTER X.
CHAPTER XI.
CHAPTER XII.
CHAPTER XIII.
CHAPTER XIV.
CHAPTER XV.
CHAPTER XVI.
CHAPTER XVII.
CHAPTER XVIII.
CHAPTER XIX.
CHAPTER XX.
CHAPTER XXI.
CHAPTER XXII.
CHAPTER XXIII.


In [99]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

In [100]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [101]:
len(all_characters), len(all_locations)

(48, 50)

In [102]:
all_characters

{'Mr. Bennet': 'Husband who is skeptical about visiting Mr. Bingley and makes sarcastic remarks',
 'Mrs. Bennet': 'Wife who is eager to marry off her daughters to Mr. Bingley',
 'Elizabeth (Lizzy) Bennet': 'She tries to reason with her mother about meeting Mr. Bingley at the assemblies.',
 'Kitty Bennet': 'She is scolded by her mother for coughing and participates in the conversation about Mr. Bingley.',
 'Lydia Bennet': 'She expresses confidence about dancing with Mr. Bingley at the next ball.',
 'Mr. Bingley': 'A lively and sociable gentleman who dances and engages with others at the ball.',
 'Mr. Darcy': 'A proud and disagreeable man who draws attention but ultimately alienates others with his demeanor.',
 'Elizabeth Bennet': "Overhears Mr. Darcy's insult and shares the story with her friends, showing her playful disposition.",
 'Jane Bennet': 'Receives admiration from Mr. Bingley and enjoys the ball quietly.',
 'Miss Bingley': "Represents the pride and conceit of Mr. Bingley's sist

In [103]:
all_locations

['Netherfield Park',
 'Bennet household',
 'Netherfield',
 'assembly room',
 'Netherfield Ball',
 'Netherfield House',
 'Longbourn',
 'Longbourn and Netherfield',
 'A drawing room or social gathering',
 'A ball',
 "Sir William Lucas's party",
 'a family gathering or social event',
 'A ballroom',
 'Pemberley',
 "Mr. Bennet's estate",
 'Longbourn and Meryton',
 "Mrs. Philips' house",
 "Mr. Bennet's home",
 'breakfast-parlour',
 'Bingley residence',
 'drawing-room',
 "Bingley's residence",
 'drawing-room at Netherfield',
 'Bennet household after morning service',
 'the study',
 'Bennet household, breakfast table',
 "Mr. Bennet's dining room",
 'Bennet household drawing-room',
 'Hunsford',
 'Meryton',
 "Mrs. Philips' home",
 'On the way home',
 "Mrs. Philips' apartment",
 'a social gathering or ball',
 'card table',
 'A conversation between Elizabeth Bennet and George Wickham',
 "Mrs. Philips's supper party",
 "Elizabeth's conversation with Jane",
 'Netherfield drawing-room',
 'a ball',
 '

In [27]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [104]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [105]:
output

AIMessage(content='{\n    "Mr. Bennet": [],\n    "Mrs. Bennet": [],\n    "Elizabeth Bennet": ["Elizabeth (Lizzy)", "Eliza (Lizzy)"],\n    "Kitty Bennet": [],\n    "Lydia Bennet": [],\n    "Mr. Bingley": ["Bingley"],\n    "Mr. Darcy": [],\n    "Jane Bennet": ["Jane"],\n    "Miss Bingley": [],\n    "Mrs. Hurst": [],\n    "Charlotte Lucas": ["Miss Lucas (Charlotte)", "Charlotte"],\n    "Mary Bennet": ["Mary"],\n    "Young Lucas": [],\n    "Sir William Lucas": [],\n    "Catherine Bennet": ["Catherine"],\n    "Aunt Gardiner": [],\n    "Mr. Philips": [],\n    "Mrs. Philips": [],\n    "Caroline Bingley": [],\n    "Mr. Hurst": [],\n    "Mr. Collins": [],\n    "Mr. Denny": [],\n    "Mr. Wickham": ["Wickham", "George Wickham"],\n    "Lady Lucas": [],\n    "Lady Catherine": [],\n    "The Bennet Family": [],\n    "Miss Bennet": [],\n    "Nieces": []\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 250, 'prompt_tokens': 1190, 'total_tokens': 1440, 'c

In [106]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [107]:
location_output

AIMessage(content='{\n    "Netherfield Park": ["Netherfield", "Netherfield House", "Netherfield Ball", "drawing-room at Netherfield", "Netherfield drawing-room", "A social gathering at Netherfield"],\n    "Bennet household": ["Bennet household after morning service", "Bennet household drawing-room", "Bennet household, breakfast table", "Mr. Bennet\'s home", "Mr. Bennet\'s estate", "A drawing room in the Bennet household", "The Bennet household"],\n    "Longbourn": ["Longbourn and Netherfield", "Longbourn and Meryton", "Longbourn estate"],\n    "Pemberley": [],\n    "Hunsford": [],\n    "Meryton": [],\n    "Lucas Lodge": ["The Lucases\' dining room"],\n    "Mrs. Philips\' house": ["Mrs. Philips\' home", "Mrs. Philips\' apartment", "Mrs. Philips\'s supper party"],\n    "Bingley residence": ["Bingley\'s residence"],\n    "A ball": ["a family gathering or social event", "a social gathering or ball", "a ballroom", "A ballroom"],\n    "assembly room": [],\n    "A conversation between Elizabe

In [108]:
# if model_type == "claude":
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [109]:
# replace characters and locations with new names
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

for scene in scenes:
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False
        if char in capitalized_output:
            character["name"] = char # capitalize the character name
            found_char = True
        for key, value in capitalized_output.items():
            # else find the key that contains the character
            for v in value:
                if char in v or v in char:
                    character["name"] = key
                    found_char = True
                    break
        if not found_char:
            character["name"] = char

    loc = string.capwords(scene["location"])
    found_loc = False
    if loc in capitalized_location_output:
        scene["location"] = loc # capitalize the location
        found_loc = True
    for key, value in capitalized_location_output.items():
        # else find the key that contains the location
        for v in value:
            if loc in v or v in loc:
                scene["location"] = key
                found_loc = True
                break
    if not found_loc:
        scene["location"] = loc

# save as json
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [110]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = character["role"]
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [111]:
len(all_characters_new), len(all_locations_new)

(32, 20)

In [112]:
all_characters_new

{'Mr. Bennet': 'Husband who is skeptical about visiting Mr. Bingley and makes sarcastic remarks',
 'Mrs. Bennet': 'Wife who is eager to marry off her daughters to Mr. Bingley',
 'Elizabeth Bennet': 'She tries to reason with her mother about meeting Mr. Bingley at the assemblies.',
 'Kitty Bennet': 'She is scolded by her mother for coughing and participates in the conversation about Mr. Bingley.',
 'Lydia Bennet': 'She expresses confidence about dancing with Mr. Bingley at the next ball.',
 'Mr. Bingley': 'A lively and sociable gentleman who dances and engages with others at the ball.',
 'Mr. Darcy': 'A proud and disagreeable man who draws attention but ultimately alienates others with his demeanor.',
 'Jane Bennet': 'Receives admiration from Mr. Bingley and enjoys the ball quietly.',
 'Mrs. Hurst': "Similar to Miss Bingley, she reflects the attitudes of Mr. Bingley's sisters.",
 'Charlotte Lucas': "Elizabeth's friend, provides insights about Mr. Bingley and Mr. Darcy.",
 'Mary Bennet':

In [113]:
all_locations_new

['Netherfield Park',
 'Bennet Household',
 'Longbourn',
 'Assembly Room',
 'A Drawing Room Or Social Gathering',
 'A Ball',
 "Sir William Lucas's Party",
 'Pemberley',
 "Mrs. Philips' House",
 'Breakfast-parlour',
 'Bingley Residence',
 'The Study',
 "Mr. Bennet's Dining Room",
 'Hunsford',
 'On The Way Home',
 'Card Table',
 'A Conversation Between Elizabeth Bennet And George Wickham',
 'Lucas Lodge',
 'Private Conference Room',
 "Elizabeth's Home"]

In [114]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [115]:
# create summary json of each chapter
chapter_summaries = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
        
    with open(f"chapters/{story_name}/{marker}.txt") as f:
        chapter = f.read()

    # get summary of chapter
    scene_summaries = " ".join([scene["summary"] for scene in scenes])
    summary = chapter_llm.invoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter),
        "num_lines": len(chapter.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }
    chapter_summaries.append(chapter_summary)
    
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
    for scene in scenes:
        chapter = scene["chapter"]
        chapter_summary = chapter_summaries[markers.index(chapter)]
        # add location to chapter summary
        location = scene["location"]
        chapter_summary_locations = chapter_summary["locations"]
        if location not in chapter_summary_locations:
            chapter_summary_locations[location] = 1
        else:
            chapter_summary_locations[location] += 1
        # add importance to chapter summary
        importance = scene["importance"]
        chapter_summary["importance"].append(importance)
        # add conflict to chapter summary
        conflict = scene["conflict"]
        chapter_summary["conflict"].append(conflict)
        # add characters to chapter summary
        characters = scene["characters"]
        for character in characters:
            char = character["name"]
            chapter_summary_characters = chapter_summary["characters"]
            if char not in chapter_summary_characters:
                chapter_summary_characters[char] = 1
            else:
                chapter_summary_characters[char] += 1

# now average importance and conflict for each chapter
for chapter_summary in chapter_summaries:
    if len(chapter_summary["importance"]) == 0:
       print(f"Chapter {chapter_summary['chapter']} has no scenes.")
    chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
    chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

# rank each chapter by importance and conflict
importances = []
conflicts = []
for chapter_summary in chapter_summaries:
    importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
    conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
    
# sort importances
sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
# add importance_rank to each chapter
for k, (j, _) in enumerate(sorted_importances):
    chapter_summaries[markers.index(j)]["importance_rank"] = k+1

# sort conflicts
sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
# add conflict_rank to each chapter
for k, (j, _) in enumerate(sorted_conflicts):
    chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

In [116]:
chapter_summaries

[{'chapter': 'CHAPTER I.',
  'scenes': 1,
  'length': 5161,
  'num_lines': 80,
  'summary': "Mr. and Mrs. Bennet discuss the arrival of Mr. Bingley and the potential benefits his wealth and status could bring to their daughters' prospects.",
  'importance': 0.9,
  'conflict': 0.5,
  'locations': {'Netherfield Park': 1},
  'characters': {'Mr. Bennet': 1, 'Mrs. Bennet': 1},
  'importance_rank': 1,
  'conflict_rank': 11},
 {'chapter': 'CHAPTER II.',
  'scenes': 1,
  'length': 4947,
  'num_lines': 76,
  'summary': "Mr. Bennet's secret visit to Mr. Bingley sparks a lively conversation about social events and introductions.",
  'importance': 0.8,
  'conflict': 0.4,
  'locations': {'Bennet Household': 1},
  'characters': {'Mr. Bennet': 1,
   'Mrs. Bennet': 1,
   'Elizabeth Bennet': 1,
   'Kitty Bennet': 1,
   'Lydia Bennet': 1},
  'importance_rank': 6,
  'conflict_rank': 20},
 {'chapter': 'CHAPTER III.',
  'scenes': 3,
  'length': 10886,
  'num_lines': 145,
  'summary': "The Bennet family lea

In [117]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
all_json["url"] = story_info["url"]
all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

In [118]:
all_json

{'title': 'Pride and Prejudice',
 'type': 'Book',
 'author': 'Jane Austen',
 'year': 1813,
 'url': 'https://www.gutenberg.org/ebooks/1342',
 'image': 'https://m.media-amazon.com/images/I/51oQBC68VSL.jpg',
 'num_chapters': 23,
 'num_scenes': 88,
 'num_characters': 32,
 'num_locations': 20,
 'chapters': [{'chapter': 'CHAPTER I.',
   'scenes': 1,
   'length': 5161,
   'num_lines': 80,
   'summary': "Mr. and Mrs. Bennet discuss the arrival of Mr. Bingley and the potential benefits his wealth and status could bring to their daughters' prospects.",
   'importance': 0.9,
   'conflict': 0.5,
   'locations': {'Netherfield Park': 1},
   'characters': {'Mr. Bennet': 1, 'Mrs. Bennet': 1},
   'importance_rank': 1,
   'conflict_rank': 11},
  {'chapter': 'CHAPTER II.',
   'scenes': 1,
   'length': 4947,
   'num_lines': 76,
   'summary': "Mr. Bennet's secret visit to Mr. Bingley sparks a lively conversation about social events and introductions.",
   'importance': 0.8,
   'conflict': 0.4,
   'location

In [119]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)